In [17]:
# See the constants defined in: https://github.com/zkcrypto/jubjub?tab=readme-ov-file

# the base field of JubJub
p = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
Fp = GF(p) 

# Edwards curve: e(x, y) = a * x^2 + y^2 - 1 - d * x^2 * y^2
ED_a = Fp(-1)
ED_d = Fp(-1) * Fp(10240) / Fp(10241)


# Montgomery curve: m(x, y) = x^3 + J * x^2 - K * y^2 + x
MONT_J = 2 * (ED_a + ED_d) / (ED_a - ED_d)
MONT_K = 4 /(ED_a - ED_d)


# Weierstrass curve: w(x, y) = x^3 - y^2 + A * x + B
WEI_A = (Fp(3) - MONT_J ** Fp(2)) / (Fp(3) * MONT_K ** Fp(2))
WEI_B = (Fp(2) * MONT_J ** Fp(3) - Fp(9) * MONT_J) / (Fp(27) * MONT_K ** Fp(3))

In [18]:
P.<x, y> = PolynomialRing(Fp, 2)
K = FractionField(P)
x, y = K.gens()

In [19]:
edwards_form = ED_a * x ** 2 + y ** 2 - 1 - ED_d * x ** 2 * y ** 2
edwards_form

33178837138445241119697427838399087845741117098273517536419461807987697107280*x^2*y^2 - x^2 + y^2 + 52435875175126190479447740508185965837690552500527637822603658699938581184512

In [20]:
montgomery_form = x * x * x + MONT_J * x ** 2 - MONT_K * y * y  + x
montgomery_form

x^3 + 40962*x^2 + 40964*y^2 + x

In [21]:
weierstrass_form = x ** 3 - y * y + WEI_A * x + WEI_B
weierstrass_form

x^3 - y^2 + 52296097456646850916096512823759002727550416093741407922227928430486925478210*x + 48351165704696163914533707656614864561753505123260775585269522553028192119009

In [22]:
# Given (x, y) in Weierstrass curve, convert to the point in Montgomery curve
def weierstrass_to_montgomery(x, y):
    return ((3 * MONT_K * x - MONT_J) / 3, y * MONT_K)

montgomery_form(weierstrass_to_montgomery(x, y)) == weierstrass_form

False

In [23]:
def montgomery_to_edwards(s, t):
    tv1 = s + 1
    tv2 = tv1 * t        # (s + 1) * t
    # tv2 = inv0(tv2)      # 1 / ((s + 1) * t)
    if tv2 != 0:
        tv2 = 1 / tv2
    v = tv2 * tv1      # 1 / t
    v = v * s          # s / t
    w = tv2 * t        # 1 / (s + 1)
    tv1 = s - 1
    w = w * tv1        # (s - 1) / (s + 1)
    # e = tv2 == 0
    # w = CMOV(w, 1, e)  # handle exceptional case
    if tv2 == 0:
        w = 1
    return (v, w)

In [24]:
E = EllipticCurve(Fp, [WEI_A, WEI_B])
P = E.random_point()
(a, b) = P.xy()

In [25]:
weierstrass_form(a, b)

0

In [26]:
(c, d) = weierstrass_to_montgomery(a, b)
montgomery_form(c, d)

0

In [27]:
(e, f) = montgomery_to_edwards(c, d)
edwards_form(e, f)

0

In [28]:
def find_z_svdw(F, A, B, init_ctr=1):
    g = lambda x: F(x)^3 + F(A) * F(x) + F(B)
    h = lambda Z: -(F(3) * Z^2 + F(4) * A) / (F(4) * g(Z))
    # NOTE: if init_ctr=1 fails to find Z, try setting it to F.gen()
    ctr = init_ctr
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            # Criterion 1:
            #   g(Z) != 0 in F.
            if g(Z_cand) == F(0):
                continue
            # Criterion 2:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) != 0 in F.
            if h(Z_cand) == F(0):
                continue
            # Criterion 3:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) is square in F.
            if not is_square(h(Z_cand)):
                continue
            # Criterion 4:
            #   At least one of g(Z) and g(-Z / 2) is square in F.
            if is_square(g(Z_cand)) or is_square(g(-Z_cand / F(2))):
                return Z_cand
        ctr += 1

In [29]:
jubjub_z = find_z_svdw(Fp, WEI_A, WEI_B)
jubjub_z

52435875175126190479447740508185965837690552500527637822603658699938581184511

In [30]:
kronecker(7, p) # i.e 7 is a quadratic non-residue

-1